In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

Tabla de posiciones La Liga 2000 - 2024

In [ ]:
""" 
    Se procede a crear una lista que contenga los años desde los cuales comenzaron las temporadas de la liga española de fútbol desde 1999 hasta 2023.
    Esta elección fue realizada para poder obtener los datos de los ganadores del torneo en el siglo XXI 
"""
temporadas = ["1999"] + [str(2000 + i) for i in range(24)]


base_clubes_url = "https://www.transfermarkt.pe/laliga/tabelle/wettbewerb/ES1/saison_id/"

dataframes_teams = []

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

In [ ]:
# Función para generar el ID de las columnas necesarias
def generadorID(tamanio_arreglo, letra):
    return [f"{letra}{id}" for id in range(tamanio_arreglo)]

In [ ]:
# Se genera una lista de IDs para las temporadas
temporadas_ID = generadorID(len(temporadas), "T")

In [ ]:
# Extracción de tabla de posiciones de La Liga: Transfermarket
for temporada in temporadas:
    url = base_clubes_url + temporada
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        df_teams = pd.read_html(response.text)[1]
        df_teams.columns = ["#","extra", "club", "partidos", "ganados", "empatados", "perdidos", "goles", "+/-", "puntos"]
        dataframes_teams.append(df_teams)

clubes_df = pd.concat(dataframes_teams, ignore_index=True)
clubes_df["golesAFavor"] = clubes_df["goles"].str.split(":").str[0]
clubes_df["golesEnContra"] = clubes_df["goles"].str.split(":").str[1]
clubes_df.drop(columns=["goles","extra"], inplace=True)
clubes_df["porcentajeVictorias"] = clubes_df["ganados"].astype(int) / clubes_df["partidos"].astype(int) * 100
clubes_df["porcentajeDerrotas"] = clubes_df["perdidos"].astype(int) / clubes_df["partidos"].astype(int) * 100
nuevo_orden = ["#", "club", "partidos", "ganados", "empatados", "perdidos", "golesAFavor", "golesEnContra", "+/-","porcentajeVictorias", "porcentajeDerrotas", "puntos"]
clubesf_df = clubes_df.reindex(columns=nuevo_orden)

In [ ]:
# Crear df de Temporada - ID

temporadas_clubes = np.repeat(temporadas_ID, 20).tolist()
temporadas_goleadores = np.repeat(temporadas_ID, 10).tolist()

inicio = 1999
fin = 2024

temporadas_df = pd.DataFrame({
    "id": temporadas_ID,
    "temporadas": temporadas
})

Goleadores de las temporadas desde 1999 hasta 2024

In [ ]:
# Extracción de goleadores de La Liga: Transfermarket
goleadores_df = []

for temporada in temporadas:
    scorers_url = f"https://www.transfermarkt.pe/laliga/torschuetzenliste/wettbewerb/ES1/saison_id/{temporada}/altersklasse/alle/detailpos//plus/1"
    response = requests.get(scorers_url, headers=headers)

    if response.status_code == 200: 
        df_players = pd.read_html(response.text)[1][:30]
        goleadores_df.append(df_players)

In [ ]:
# Limpiar y renombrar columnas
goleadoresf_df = pd.concat(goleadores_df, ignore_index=True)
goleadoresf_df.columns = ["#", "jugador", "nacionalidad", "edad(Hoy)", "club", 
                            "partidosJugados", "asistencias", "penales", "minutosJugados", 
                            "minutosPorGol", "golesPorPartido", "goles"]
goleadoresf_df = goleadoresf_df[goleadoresf_df["#"].notna()]
goleadoresf_df.index = np.arange(len(goleadoresf_df))

In [ ]:
# Posiciones de jugadores identificadas manualmente
lista_posiciones = ['Mediapunta','Interior', 'Mediocentro', 'Delantero centro', 'Delantero','Extremo derecho', 'Mediocentro ofensivo', 'Extremo','Extremo izquierdo',]

In [ ]:
# Limpieza y separación de nombres y posiciones

series_nombres = []
series_posiciones = []

for jugador in goleadoresf_df["jugador"].values:
    found = False
    for pos in jugador.split():
        if pos in lista_posiciones:
            index = jugador.find(pos)
            series_nombres.append(jugador[:index].strip())
            series_posiciones.append(jugador[index:].strip())
            found = True
            break

goleadoresf_df["nombre"] = series_nombres
goleadoresf_df["posición"] = series_posiciones

In [ ]:
goleadoresf_df = goleadoresf_df.drop(columns=["jugador"])

In [ ]:
# Limpieza de datos
arr_minutos = []

for i in goleadoresf_df["minutosJugados"].values:
    arr_minutos.append(float(i.replace(".","").replace("'","")))
goleadoresf_df["minutosJugados"] = arr_minutos

arr_minutos_gol = []

for i in goleadoresf_df["minutosPorGol"].values:
    arr_minutos_gol.append(float(i.replace("'","")))
goleadoresf_df["minutosPorGol"] = arr_minutos_gol

goleadoresf_df["#"] = goleadoresf_df["#"].astype(int)

goleadoresf_df["partidosJugados"] = goleadoresf_df["partidosJugados"].astype(int)

goleadoresf_df["asistencias"] = goleadoresf_df["asistencias"].astype(int)

goleadoresf_df["penales"] = goleadoresf_df["penales"].astype(int)

In [ ]:
# Extracción de países y clubes de los goleadores mediante beautifulsoup
arr_paises = []
arr_clubes_jugador = []

for temporada in temporadas:
    url_pais = f"https://www.transfermarkt.pe/laliga/torschuetzenliste/wettbewerb/ES1/saison_id/{temporada}/altersklasse/alle/detailpos//plus/1"
    response = requests.get(url_pais, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    tabla = soup.find("table", {"class": "items"})
    tabla_body = tabla.find("tbody")
    
    for i in range(10):
        tabla_rows = tabla_body.find_all("tr",{"class":["odd","even"]})[i]
        tabla_cols_paises = tabla_rows.find_all("td",{"class":"zentriert"})[1]
        tabla_cols_club = tabla_rows.find_all("td",{"class":"zentriert"})[3]

        img = tabla_cols_paises.find("img")
        pais = img["title"]
        club = tabla_cols_club.find("a")["title"]

        arr_paises.append(pais.strip())
        arr_clubes_jugador.append(club.strip())
        
goleadoresf_df["nacionalidad"] = arr_paises
goleadoresf_df["club"] = arr_clubes_jugador


In [ ]:
# Limpieza de clubes para igualar los nombres:
goleadoresf_df["club"] = goleadoresf_df["club"].str.replace("Real Racing Club", "Racing")
goleadoresf_df["club"] = goleadoresf_df["club"].str.replace("Atlético de Madrid", "Atlético Madrid")
goleadoresf_df["club"] = goleadoresf_df["club"].str.replace("RC Deportivo de La Coruña", "RC Deportivo")
goleadoresf_df["club"] = goleadoresf_df["club"].str.replace("Real Madrid CF", "Real Madrid")
goleadoresf_df["club"] = goleadoresf_df["club"].str.replace("Real Valladolid CF", "Real Valladolid")
goleadoresf_df["club"] = goleadoresf_df["club"].str.replace("Deportivo Alavés", "Alavés")
goleadoresf_df["club"] = goleadoresf_df["club"].str.replace("RC Celta de Vigo", "RC Celta")
goleadoresf_df["club"] = goleadoresf_df["club"].str.replace("Athletic Club", "Athletic")
goleadoresf_df["club"] = goleadoresf_df["club"].str.replace("Real Betis Balompié", "Real Betis")


In [ ]:
# Creación de ID para clubes
arr_clubes = sorted(set(list(clubesf_df["club"].unique()) + list(goleadoresf_df["club"].unique())))
club_id = generadorID(len(arr_clubes), "C")

dict_clubes = dict(zip(arr_clubes,club_id))

In [ ]:
# Mapeo de clubes a IDs en los dataframes
clubesf_df["idClub"] = clubesf_df["club"].map(dict_clubes)
clubesf_df["idTemporada"] = temporadas_clubes
df_final_orden = ["idTemporada", "#", "club", "idClub", "partidos", "ganados", "empatados", "perdidos", "golesAFavor", "golesEnContra", "+/-","porcentajeVictorias", "porcentajeDerrotas", "puntos"]
clubesf_df = clubesf_df[df_final_orden]

# Mapeo de goleadores a IDs en los dataframes
goleadoresf_df["idClub"] = goleadoresf_df["club"].map(dict_clubes)
goleadoresf_df["idTemporada"] = temporadas_goleadores
df_jugador_orden = ["idTemporada", "#", "nombre", "posición", "nacionalidad", "club", "idClub", "partidosJugados", "asistencias", "penales", "minutosJugados", "minutosPorGol", "golesPorPartido", "goles"]
goleadoresf_df = goleadoresf_df[df_jugador_orden]


In [ ]:
clubes_id_df = pd.DataFrame(dict_clubes.items(), columns=["club", "id"])
orden_clubes = ["id", "club"]
clubes_id_df = clubes_id_df.reindex(columns=orden_clubes)

In [ ]:
# Convertimos todos los dataframes a CSV

clubesf_df.to_csv("clubes_liga.csv", index=False, encoding="utf-8")
goleadoresf_df.to_csv("goleadores_liga.csv", index=False, encoding="utf-8")
temporadas_df.to_csv("temporadas_liga.csv", index=False, encoding="utf-8")
clubes_id_df.to_csv("clubes_id_liga.csv", index=False, encoding="utf-8")